Alright, today I'm going to be going through the process of building a price and fundamental database for equitites. In future I'm going to be using this as my primary source of data for my equity trading research and live operations. I'm using [Sharadar](https://www.quandl.com/databases/SFA/data) from Quandl for all of the data. It's relatively affordable, updated daily, and includes delisted tickers going back decades. The linked bundle also includes fund prices, institutional investor, and insider investor information. Quandl is a really fantastic platform that is well supported, so I'm looking forward to using it!

### Intial Database Setup

---


I'm going to use SQLite for this database. The reasoning for this is that I only need single user access, and it's much simpler to setup and maintain than something like MySQL or PostgreSQL. I don't require much fancy SQL for what I'm doing either.

In [1]:
import quandl
import pandas as pd
# Read API Key from file. See Quandl Docs.
quandl.read_key()

In [2]:
# Fetch ticker information for each table
fundamental_tickers = quandl.get_table('SHARADAR/TICKERS', table='SF1', paginate=True)
price_tickers = quandl.get_table('SHARADAR/TICKERS', table='SEP', paginate=True)
fund_tickers = quandl.get_table('SHARADAR/TICKERS', table='SFP', paginate=True)

# This combines them all together and drops duplicates, so I can get a single table
# containing all tickers that I have data for
tickers_frame = pd.concat([fundamental_tickers, price_tickers, fund_tickers])
tickers_frame = tickers_frame.drop_duplicates('permaticker')

In [3]:
# An example of what I get
tickers_frame.head()

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic Common Stock,00846U101,3826.0,Manufacturing,...,USD,California; U.S.A,2020-12-18,2014-09-26,1999-11-18,2021-01-22,1997-06-30,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic Common Stock,013872106,3334.0,Manufacturing,...,USD,Pennsylvania; U.S.A,2020-10-30,2016-11-01,2016-11-01,2021-01-22,2014-12-31,2020-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,122827,AAAB,Admiralty Bancorp Inc,NASDAQ,Y,Domestic Common Stock,007231103,6022.0,Finance Insurance And Real Estate,...,USD,Florida; U.S.A,2019-07-29,2017-09-09,1998-09-25,2003-01-28,1997-09-30,2002-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
3,SF1,120538,AAAGY,Altana Aktiengesellschaft,NYSE,Y,ADR Common Stock,02143N103,2834.0,Manufacturing,...,EUR,Jordan,2019-05-17,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
4,SF1,155760,AAAP,Advanced Accelerator Applications SA,NASDAQ,Y,ADR Common Stock,00790T100,2834.0,Manufacturing,...,EUR,France,2020-10-08,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2017-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None


In [2]:
# Setup our SQLite database connection
import sqlite3
conn = sqlite3.connect('data.db')

In [5]:
# Let's create the table to hold the tickers data!
with conn:
    conn.execute('''
        CREATE TABLE tickers(
        permaticker NUMERIC PRIMARY KEY,
        ticker TEXT,
        name TEXT,
        exchange TEXT,
        isdelisted TEXT,
        category TEXT,
        cusips TEXT,
        siccode NUMERIC,
        sicsector TEXT,
        sicindustry TEXT,
        famasector TEXT,
        famaindustry TEXT,
        sector TEXT,
        industry TEXT,
        scalemarketcap TEXT,
        scalerevenue TEXT,
        relatedtickers TEXT,
        currency TEXT,
        location TEXT,
        lastupdated TEXT,
        firstadded TEXT,
        firstpricedate TEXT,
        lastpricedate TEXT,
        firstquarter TEXT,
        lastquarter TEXT,
        secfilings TEXT,
        companysite TEXT
        )
    ''')

OperationalError: table tickers already exists

In [6]:
# This is the order that the data needs to be in so the database gets filled correctly. This becomes relevant if the
# order of the columns in the data we get from the API changes at some point down the line.
column_order = ['permaticker', 'ticker', 'name', 'exchange', 'isdelisted',
                'category', 'cusips', 'siccode', 'sicsector', 'sicindustry',
                'famasector', 'famaindustry', 'sector', 'industry', 'scalemarketcap',
                'scalerevenue', 'relatedtickers', 'currency', 'location', 'lastupdated',
                'firstadded', 'firstpricedate', 'lastpricedate', 'firstquarter',
                'lastquarter', 'secfilings', 'companysite']

# Now let's insert that ticker data. Let's first select all the data we need in the right order
tickers_frame = tickers_frame[column_order]
# This converts the dataframe into a list of tuples where each tuple is a row of the dataframe.
# As a helpful device, I convert all data to string and let SQLite handle converting the datatypes.
# If I don't do this I run into all kinds of problems converting from python types to the proper types for
# SQLite.
tickers_records = tickers_frame.astype('str').to_records(index=False)

In [7]:
# Like this
tickers_records[0]

('196290', 'A', 'Agilent Technologies Inc', 'NYSE', 'N', 'Domestic Common Stock', '00846U101', '3826.0', 'Manufacturing', 'Laboratory Analytical Instruments', 'None', 'Measuring and Control Equipment', 'Healthcare', 'Diagnostics & Research', '5 - Large', '5 - Large', 'None', 'USD', 'California; U.S.A', '2020-12-18', '2014-09-26', '1999-11-18', '2021-01-22', '1997-06-30', '2020-09-30', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001090872', 'http://www.agilent.com')

In [8]:
with conn:
    conn.executemany('''
        INSERT INTO tickers VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tickers_records)

IntegrityError: UNIQUE constraint failed: tickers.permaticker

Alright, so now we have all the tickers in a table that we have data for. Next step is getting that data inserted!

In [9]:
# First let's make a table to store the data
with conn:
    conn.execute('''
        CREATE TABLE prices(
            ticker REFERENCES tickers(ticker),
            date TEXT,
            open NUMERIC,
            high NUMERIC,
            low NUMERIC,
            close NUMERIC,
            volume NUMERIC,
            dividends NUMERIC,
            closeunadj NUMERIC,
            lastupdated NUMERIC,
            PRIMARY KEY (ticker, date)
        )
    ''')

In [10]:
# Same as above, select data in proper order
column_order = ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'closeunadj', 'lastupdated']
# I bulk downloaded the table, so I need to read that from CSV. Same as above, make all data type string and create records
sfp_data = pd.read_csv('./SHARADAR_SFP.csv', dtype='str')[column_order].to_records(index=False)

In [13]:
# And insert all the data
with conn:
    conn.executemany('''
        INSERT INTO prices VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sfp_data)

In [15]:
# Now for the equity prices data
sep_data = pd.read_csv('./SHARADAR_SEP.csv', dtype='str')[column_order].to_records(index=False)

In [17]:
# And insert all the data
with conn:
    conn.executemany('''
        INSERT INTO prices VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sep_data)

Next up is the fundamental data.

In [3]:
# First, make the table
with conn:
    conn.execute('''
        CREATE TABLE fundamentals(
            ticker TEXT REFERENCES tickers(ticker),
            dimension TEXT,
            calendardate TEXT,
            datekey TEXT,
            reportperiod TEXT,
            lastupdated TEXT,
            accoci NUMERIC,
            assets NUMERIC,
            assetsavg NUMERIC,
            assetsc NUMERIC,
            assetsnc NUMERIC,
            assetturnover NUMERIC,
            bvps NUMERIC,
            capex NUMERIC,
            cashneq NUMERIC,
            cashnequsd NUMERIC,
            cor NUMERIC,
            consolinc NUMERIC,
            currentratio NUMERIC,
            de NUMERIC,
            debt NUMERIC,
            debtc NUMERIC,
            debtnc NUMERIC,
            debtusd NUMERIC,
            deferredrev NUMERIC,
            depamor NUMERIC,
            deposits NUMERIC,
            divyield NUMERIC,
            dps NUMERIC,
            ebit NUMERIC,
            ebitda NUMERIC,
            ebitdamargin NUMERIC,
            ebitdausd NUMERIC,
            ebitusd NUMERIC,
            ebt NUMERIC,
            eps NUMERIC,
            epsdil NUMERIC,
            epsusd NUMERIC,
            equity NUMERIC,
            equityavg NUMERIC,
            equityusd NUMERIC,
            ev NUMERIC,
            evebit NUMERIC,
            evebitda NUMERIC,
            fcf NUMERIC,
            fcfps NUMERIC,
            fxusd NUMERIC,
            gp NUMERIC,
            grossmargin NUMERIC,
            intangibles NUMERIC,
            intexp NUMERIC,
            invcap NUMERIC,
            invcapavg NUMERIC,
            inventory NUMERIC,
            investments NUMERIC,
            investmentsc NUMERIC,
            investmentsnc NUMERIC,
            liabilities NUMERIC,
            liabilitiesc NUMERIC,
            liabilitiesnc,
            marketcap NUMERIC,
            ncf NUMERIC,
            ncfbus NUMERIC,
            ncfcommon NUMERIC,
            ncfdebt NUMERIC,
            ncfdiv NUMERIC,
            ncff NUMERIC,
            ncfi NUMERIC,
            ncfinv NUMERIC,
            ncfo NUMERIC,
            ncfx NUMERIC,
            netinc NUMERIC,
            netinccmn NUMERIC,
            netinccmnusd NUMERIC,
            netincdis NUMERIC,
            netincnci NUMERIC,
            netmargin NUMERIC,
            opex NUMERIC,
            opinc NUMERIC,
            payables NUMERIC,
            payoutratio NUMERIC,
            pb NUMERIC,
            pe NUMERIC,
            pe1 NUMERIC,
            ppnenet NUMERIC,
            prefdivis NUMERIC,
            price NUMERIC,
            ps NUMERIC,
            ps1 NUMERIC,
            receivables NUMERIC,
            retearn NUMERIC,
            revenue NUMERIC,
            revenueusd NUMERIC,
            rnd NUMERIC,
            roa NUMERIC,
            roe NUMERIC,
            roic NUMERIC,
            ros NUMERIC,
            sbcomp NUMERIC,
            sgna NUMERIC,
            sharefactor NUMERIC,
            sharesbas NUMERIC,
            shareswa NUMERIC,
            shareswadil NUMERIC,
            sps NUMERIC,
            tangibles NUMERIC,
            taxassets NUMERIC,
            taxexp NUMERIC,
            taxliabilities NUMERIC,
            tbvps NUMERIC,
            workingcapital NUMERIC,
            PRIMARY KEY (ticker, dimension, datekey)
        )
    ''')

In [4]:
# Select columns in right order, same as above
column_order = ['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod', 'lastupdated', 'accoci', 'assets',
                'assetsavg', 'assetsc', 'assetsnc', 'assetturnover', 'bvps', 'capex', 'cashneq', 'cashnequsd', 'cor',
                'consolinc', 'currentratio', 'de', 'debt', 'debtc', 'debtnc', 'debtusd', 'deferredrev', 'depamor', 'deposits',
                'divyield', 'dps', 'ebit', 'ebitda', 'ebitdamargin', 'ebitdausd', 'ebitusd', 'ebt', 'eps', 'epsdil', 'epsusd',
                'equity', 'equityavg', 'equityusd', 'ev', 'evebit', 'evebitda', 'fcf', 'fcfps', 'fxusd', 'gp', 'grossmargin',
                'intangibles', 'intexp', 'invcap', 'invcapavg', 'inventory', 'investments', 'investmentsc', 'investmentsnc',
                'liabilities', 'liabilitiesc', 'liabilitiesnc', 'marketcap', 'ncf', 'ncfbus', 'ncfcommon', 'ncfdebt',
                'ncfdiv', 'ncff', 'ncfi', 'ncfinv', 'ncfo', 'ncfx', 'netinc', 'netinccmn', 'netinccmnusd', 'netincdis',
                'netincnci', 'netmargin', 'opex', 'opinc', 'payables', 'payoutratio', 'pb', 'pe', 'pe1', 'ppnenet',
                'prefdivis', 'price', 'ps', 'ps1', 'receivables', 'retearn', 'revenue','revenueusd', 'rnd', 'roa',
                'roe', 'roic', 'ros', 'sbcomp', 'sgna', 'sharefactor', 'sharesbas', 'shareswa', 'shareswadil',
                'sps', 'tangibles', 'taxassets', 'taxexp', 'taxliabilities', 'tbvps', 'workingcapital']
sf1_data = pd.read_csv('./SHARADAR_SF1.csv', dtype='str')[column_order].to_records(index=False)

In [5]:
# And do insert
with conn:
    conn.executemany('''
        INSERT INTO fundamentals VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sf1_data)

### Database Updates

----


Okay, now we have the bulk of the historical data written in, I need to ensure that I can keep that data up to date.

In [19]:
# Okay, let's select the most recent 'lastupdated' date from the prices table
with conn:
    execute = conn.execute('''
        SELECT MAX(lastupdated) FROM prices;
    ''')
    
date = execute.fetchone()[0]

In [38]:
# Fetch recently updated tickers. Pretty much follows the original insert
fundamental_tickers = quandl.get_table('SHARADAR/TICKERS', table='SF1', paginate=True, lastupdated={'gt': date})
price_tickers = quandl.get_table('SHARADAR/TICKERS', table='SEP', paginate=True, lastupdated={'gt': date})
fund_tickers = quandl.get_table('SHARADAR/TICKERS', table='SFP', paginate=True, lastupdated={'gt': date})

tickers_frame = pd.concat([fundamental_tickers, price_tickers, fund_tickers])
tickers_frame = tickers_frame.drop_duplicates('permaticker')

column_order = ['permaticker', 'ticker', 'name', 'exchange', 'isdelisted',
                'category', 'cusips', 'siccode', 'sicsector', 'sicindustry',
                'famasector', 'famaindustry', 'sector', 'industry', 'scalemarketcap',
                'scalerevenue', 'relatedtickers', 'currency', 'location', 'lastupdated',
                'firstadded', 'firstpricedate', 'lastpricedate', 'firstquarter',
                'lastquarter', 'secfilings', 'companysite']

tickers_frame = tickers_frame[column_order]
tickers_records = tickers_frame.astype('str').to_records(index=False)

with conn:
    conn.executemany('''
        INSERT OR REPLACE INTO tickers VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tickers_records)

In [40]:
# Update fund prices data
quandl.export_table('SHARADAR/SFP', filename=filename, lastupdated={'gt': date})

data_zip = zipfile.ZipFile(filename)
csv_name = data_zip.namelist()[0]
csv_data = data_zip.open(csv_name)

column_order = ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'closeunadj', 'lastupdated']
sfp_records = pd.read_csv(csv_data)[column_order].to_records(index=False)

with conn:
    conn.executemany('''
        INSERT OR REPLACE INTO prices VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sfp_records)

In [20]:
# Depending upon how long its been since the last update, the return can be quite large, so I need to mkae
# a bulk request as a ZIP download
filename = 'recent_updates.zip'
quandl.export_table('SHARADAR/SEP', filename=filename, lastupdated={'gt': date})

import zipfile
# Extract the csv data from the downloaded ZIP file
data_zip = zipfile.ZipFile(filename)
csv_name = data_zip.namelist()[0]
csv_data = data_zip.open(csv_name)

# And now I can read and select the data same as above
column_order = ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'closeunadj', 'lastupdated']
sep_records = pd.read_csv(csv_data)[column_order].to_records(index=False)

# And write into the database! We use INSERT OR REPLACE so that the row gets updated when there is a primary
# key conflict
with conn:
    conn.executemany('''
        INSERT OR REPLACE INTO prices VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sep_records)

In [49]:
# Update fundmanental data
# Get most recent update for fundamentals table
with conn:
    execute = conn.execute('''SELECT MAX(lastupdated) FROM fundamentals''')
    
date = execute.fetchone()[0]

quandl.export_table('SHARADAR/SF1', filename=filename, lastupdated={'gt': date})

data_zip = zipfile.ZipFile(filename)
csv_name = data_zip.namelist()[0]
csv_data = data_zip.open(csv_name)

column_order = ['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod', 'lastupdated', 'accoci', 'assets',
                'assetsavg', 'assetsc', 'assetsnc', 'assetturnover', 'bvps', 'capex', 'cashneq', 'cashnequsd', 'cor',
                'consolinc', 'currentratio', 'de', 'debt', 'debtc', 'debtnc', 'debtusd', 'deferredrev', 'depamor', 'deposits',
                'divyield', 'dps', 'ebit', 'ebitda', 'ebitdamargin', 'ebitdausd', 'ebitusd', 'ebt', 'eps', 'epsdil', 'epsusd',
                'equity', 'equityavg', 'equityusd', 'ev', 'evebit', 'evebitda', 'fcf', 'fcfps', 'fxusd', 'gp', 'grossmargin',
                'intangibles', 'intexp', 'invcap', 'invcapavg', 'inventory', 'investments', 'investmentsc', 'investmentsnc',
                'liabilities', 'liabilitiesc', 'liabilitiesnc', 'marketcap', 'ncf', 'ncfbus', 'ncfcommon', 'ncfdebt',
                'ncfdiv', 'ncff', 'ncfi', 'ncfinv', 'ncfo', 'ncfx', 'netinc', 'netinccmn', 'netinccmnusd', 'netincdis',
                'netincnci', 'netmargin', 'opex', 'opinc', 'payables', 'payoutratio', 'pb', 'pe', 'pe1', 'ppnenet',
                'prefdivis', 'price', 'ps', 'ps1', 'receivables', 'retearn', 'revenue','revenueusd', 'rnd', 'roa',
                'roe', 'roic', 'ros', 'sbcomp', 'sgna', 'sharefactor', 'sharesbas', 'shareswa', 'shareswadil',
                'sps', 'tangibles', 'taxassets', 'taxexp', 'taxliabilities', 'tbvps', 'workingcapital']
sf1_records = pd.read_csv(csv_data, dtype='str')[column_order].to_records(index=False)

with conn:
    conn.executemany('''
        INSERT OR REPLACE INTO fundamentals VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sf1_records)

In [64]:
# Delete zip file
import os
os.remove(filename)